### 4.4 对数几率回归

In [3]:
import tensorflow as tf
import os

# w1, w2, ..., wk为模型从训练数据中学习到的参数，或赋予每个变量的“权值”
W = tf.Variable(tf.zeros([5,1]), name="weights")
# b也是一个学习到的参数，这个线性代数中的常量也称为模型的偏置（bias）
b = tf.Variable(0., name="bias")

# 初始化变量和模型参数，定义训练闭环中的运算


def combine_inputs(X):
    return tf.matmul(X, W) + b
def inference(X):
    # 计算推断模型在数据X上的输出，并将结果返回
    return tf.sigmoid(combine_inputs(X))
def loss(X, Y):
    # 依据训练数据X及其期望输出Y计算损失
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=combine_inputs(X)))
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.getcwd() + '/data/www_kaggle_com_c_titanic_data/' + file_name])
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    # decode_csv会将字符串（文本行）转换到具有制定默认值的由张量列构成的元组中
    # 它还会为每一列设置数据类型
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    
    # 实际上会读取一个文件，并加载一个张量中的batch_size行
    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size*50, min_after_dequeue=batch_size)
def inputs():
    # 读取或生成训练数据X及其期望输出Y
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, far, cabin, embarked = \
    read_csv(100, "train.csv",[[0.0],[0.0],[0],[""],[""],[0.0],[0.0],[0.0],[""],[0.0],[""],[""]])
    
    # 转换属性数据
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    
    gender = tf.to_float(tf.equal(sex, ["female"]))
    
    # 最终将所有特征排列在一个矩阵中，然后对该矩阵转置，使其每行对应一个样本，每列对应一种特征
    features = tf.transpose(tf.stack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])
    return features, survived
def train(total_loss):
    # 依据计算的总损失训练或调整模型参数
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
def evaluate(sess, X, Y):
    # 对训练得到的模型进行评估
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))
    
# 在一个会话对象中启动数据流图，搭建流程
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 实际的训练迭代次数
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # 出于调试和学习的目的，查看损失在训练过程中递减的情况
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
            
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [0.64989424]
loss:  [0.69331527]
loss:  [0.92895836]
loss:  [0.66836488]
loss:  [0.74244505]
loss:  [0.69695085]
loss:  [0.67592955]
loss:  [0.72575921]
loss:  [0.67585081]
loss:  [0.6735208]
loss:  [0.72624451]
loss:  [0.64427042]
loss:  [0.64188737]
loss:  [0.70129591]
loss:  [0.68212831]
loss:  [0.69905871]
loss:  [0.6691252]
loss:  [0.66264921]
loss:  [0.69183326]
loss:  [0.72818077]
loss:  [1.0904338]
loss:  [0.58406985]
loss:  [0.67048568]
loss:  [0.60695797]
loss:  [0.66281438]
loss:  [0.71037722]
loss:  [0.64567071]
loss:  [0.61532015]
loss:  [0.85648072]
loss:  [0.61714751]
loss:  [0.59250975]
loss:  [0.60139388]
loss:  [0.62885481]
loss:  [0.77748984]
loss:  [0.56243843]
loss:  [0.62095952]
loss:  [0.74416089]
loss:  [0.63972384]
loss:  [0.56708288]
loss:  [0.59830731]
loss:  [0.79444093]
loss:  [0.67639434]
loss:  [0.61746037]
loss:  [0.68473297]
loss:  [0.70888156]
loss:  [0.66106033]
loss:  [0.59290487]
loss:  [0.57953542]
loss:  [0.61517137]
loss:  [0.81705445]
los